In [69]:
from ultralytics import YOLO
from sahi.predict import get_sliced_prediction, AutoDetectionModel
import cv2
import supervision as sv
from PIL import Image

In [82]:
image_path = "people5.jpg"
image = cv2.imread(image_path)


detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path="yolov8n.pt",
    confidence_threshold=0.3,
    device="cpu"
)


In [85]:
#results = model.predict(image, conf=0.1, classes=[0])[0]
result = get_sliced_prediction(
    image=image, 
    detection_model=detection_model,
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.5,
    overlap_width_ratio=0.5,
    verbose=1
)


people_predictions = [
    p for p in result.object_prediction_list
    if p.category.id == 0
]

print(f"Detected {len(people_predictions)} people.")




Performing prediction on 126 number of slices.
Detected 79 people.


In [86]:
# Each prediction p has attributes: bbox, score, category.id/name
xyxy = []
confidences = []
class_ids = []

for p in people_predictions:
    x1, y1, x2, y2 = p.bbox.to_xyxy()
    xyxy.append([x1, y1, x2, y2])
    confidences.append(p.score.value)
    class_ids.append(p.category.id)

import numpy as np
xyxy = np.array(xyxy, dtype=float)          # shape (N,4)
confidences = np.array(confidences, dtype=float)
class_ids = np.array(class_ids, dtype=int)

detections = sv.Detections(
    xyxy=xyxy,
    confidence=confidences,
    class_id=class_ids
)

# --- Annotate and save result ---
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated = box_annotator.annotate(image.copy(), detections)
annotated = label_annotator.annotate(annotated, detections)

cv2.imwrite("results/people_detection.png", annotated)
print("Annotated image saved to results/people_detection.png")

# Optional: display result with PIL
Image.open("results/people_detection.png").show()

Annotated image saved to results/people_detection.png
